In [1]:
import igl
import numpy as np
import math
import random as rnd
import scipy as sp
from scipy.spatial import KDTree
import meshplot as mp
from meshplot import plot, subplot, interact

In [3]:
from datetime import datetime

In [4]:
start = datetime.now()
v, f = igl.read_triangle_mesh("data/30_original.off") #("3dPotteryDs/3D Models/Lagynos/qpvase5.obj")
end = datetime.now()
print(end - start)

0:00:03.037137


In [5]:
start = datetime.now()

n_vertex = igl.per_vertex_normals(v, f)
k = igl.gaussian_curvature(v, f)
kn = k[(~np.isnan(k)) & (k < 2*math.pi)]
#v1, v2, k1, k2 = igl.principal_curvature(v, f)
#v1_3, v2_3, k1_3, k2_3 = igl.principal_curvature(v, f, 3, True)

#Returns list of lists containing at index i the adjacent vertices of vertex i
adj_lst = igl.adjacency_list(f) 

#Returns
#VF (size: 3*#F) List of faces indice on each vertex, so that VF(NI(i)+j) = f, means that face f is the jth face 
# (in no particular order) incident on vertex i.
#VFi (size: #V+1) cumulative sum of vertex-triangle degrees with a preceeding zero. 
# “How many faces” have been seen before visiting this vertex and its incident faces.
VF, VFi = igl.vertex_triangle_adjacency(f, len(v))
bv = igl.is_border_vertex(v, f)

end = datetime.now()
print(end - start)

0:00:01.650989


In [6]:
#Calculate curvatures for sampled points
area_t = 0.5 * sum(igl.doublearea(v, f))
v_size = 3 * (v.shape[0]**(1/2)) #min(round(area_t/4), round(v.shape[0] / 100))
v_ratio = math.floor(v.shape[0] / v_size)
v_size = math.floor(v.shape[0] / v_ratio)
round(area_t, 4), v.shape[0], v_ratio, v_size, v_ratio * v_size, len(bv)

(40107.8031, 508963, 237, 2147, 508839, 508962)

In [7]:
def calculate_principal(ivx): #curvature with 3-rounds
    lfaces, lvertices = [], []
    lvertices.append(ivx)
    lvx_r1, lvx_r2 = [], []

    for fx in VF[VFi[ivx]:VFi[ivx+1]]:
        lfaces.append(fx) #1-round faces

    for vr1 in adj_lst[ivx]:
        if vr1 not in lvertices:
            lvertices.append(vr1) #1-round vertices
            lvx_r1.append(vr1)
            
    for vr1 in lvx_r1:
        for fx2 in VF[VFi[vr1]:VFi[vr1+1]]:
            if fx2 not in lfaces:
                lfaces.append(fx2)
                                
        for vr2 in adj_lst[vr1]:
            if vr2 not in lvertices:
                lvertices.append(vr2)
                lvx_r2.append(vr2)
    
    for vr2 in lvx_r2:
        for fx3 in VF[VFi[vr2]:VFi[vr2+1]]:
            if fx3 not in lfaces:
                lfaces.append(fx3)

        for vr3 in adj_lst[vr2]:
            if vr3 not in lvertices:
                lvertices.append(vr3)

    dtv = {} #vertex dictionary
    for ix, vertex in enumerate(lvertices):
        dtv[vertex] = ix    
    
    vc = np.array([v[lvertices[i]] for i in range(len(lvertices))])
    fc = np.array([[dtv[f[lfaces[i]][0]], dtv[f[lfaces[i]][1]], dtv[f[lfaces[i]][2]]] for i in range(len(lfaces))], dtype=int)
    v1c, v2c, k1c, k2c = igl.principal_curvature(vc, fc)
    return v1c[0], v2c[0], k1c[0], k2c[0]

In [8]:
start = datetime.now()
v_indices = [] #np.array([0 for i in range(v_size)])
v1_p, v2_p, k1_p, k2_p = [], [], [], []
k_p = np.percentile(abs(kn), 33)

from datetime import datetime
start = datetime.now()

for i in range(0, v_size):
    while True:
        index = rnd.randint(v_ratio * i, min(v_ratio * (i + 1) - 1, v.shape[0] - 1))
        if len(adj_lst[index]) > 0:
            break            
    
    adj_neighbors = False
    
    if not bv[index]:
        adj_neighbors = True
        
        for nb in adj_lst[index]:
            if len(adj_lst[nb]) == 0:
                adj_neighbors = False
        
    if adj_neighbors and abs(k[index]) > k_p:
        v_indices.append(index)        
        v1p, v2p, k1p, k2p = calculate_principal(index)
        v1_p.append(v1p)
        v2_p.append(v2p)
        k1_p.append(k1p * np.sign(k2p))
        k2_p.append(k2p * np.sign(k2p))

end = datetime.now()
print(end - start)

0:00:03.878723


In [9]:
oVertex = []
v1_c, v2_c, k1_c, k2_c = [], [], [], []

for j in range(0, len(v_indices)):
    if (abs(k1_p[j]) > np.percentile(np.abs(k1_p), 50)) or (abs(k2_p[j]) > np.percentile(np.abs(k2_p), 50)):
        #print(v_indices[j], k1_c[j], k2_c[j], k_c[v_indices[j]])
        oVertex.append(v_indices[j])
        v1_c.append(v1_p[j])
        v2_c.append(v2_p[j])
        k1_c.append(k1_p[j])
        k2_c.append(k2_p[j])

len(v_indices), len(oVertex)

(1390, 1059)

In [10]:
TH_RATIO_K = 0.9
TH_COSINE = 0.75

#distance / bounding_box, angle
CLUSTER_EP = [0.125, 0.125] #0.125

In [11]:
length_x = np.percentile(v[1:,0], 99.9) - np.percentile(v[1:,0], 0.1)
length_y = np.percentile(v[1:,1], 99.9) - np.percentile(v[1:,1], 0.1)
length_z = np.percentile(v[1:,2], 99.9) - np.percentile(v[1:,2], 0.1)

diagonal_bbox = math.sqrt(length_x * length_x + length_y * length_y + length_z * length_z)
diagonal_bbox, length_x, length_y, length_z

(221.52662417560558, 142.7342, 103.81300000000002, 133.88)

In [12]:
def projectPointPlane(point, plane, planeD):
    k = point.dot(plane) + planeD
    return point - k * plane

def distanceMetricRx(rxVectorA, rxProjA, rxVectorB, rxProjB):
    module = np.linalg.norm(rxVectorA) * np.linalg.norm(rxVectorB)
    angle = math.acos(min(1.0, abs(np.dot(rxVectorA, rxVectorB)) / module))
    
    origin = np.array([0, 0, 0])
    #projectionA = projectPointPlane(origin, rxPairA[2], rxPairA[3])
    #projectionB = projectPointPlane(origin, rxPairB[2], rxPairB[3])
    distance = np.linalg.norm(rxProjA - rxProjB)
    
    return distance / diagonal_bbox, angle

In [13]:
# iPoints : indexes list of points Sampled (KdTree)
def baseQReflection(iPoints): 
    sample_size = len(iPoints)
    lxPairs = []
    
    lk1 = np.percentile(k1_c, 99.5) - np.percentile(k1_c, 0.5)
    lk2 = np.percentile(k2_c, 99.5) - np.percentile(k2_c, 0.5)
    rd = np.sqrt(lk1*lk1 + lk2*lk2) / 16
    kl = [[k1_c[i], k2_c[i]] for i in range(len(k1_c))]
    tk = KDTree(kl)

    for i in range(0, sample_size - 1):
        idx = tk.query_ball_point([k1_c[i], k2_c[i]], r = rd, return_sorted=True)
        idl = [kx for kx in idx if kx > i]
        
        for j in idl: #range(i + 1, sample_size):
            ratio_1, ratio_2 = k1_c[i] / k1_c[j], k2_c[i] / k2_c[j]
            ix, jx = iPoints[i], iPoints[j]
            
            if TH_RATIO_K < ratio_1 < 1/TH_RATIO_K and TH_RATIO_K < ratio_2 < 1/TH_RATIO_K: #ratio_1 * ratio_2 > 0
                reflexNormal = v[ix] - v[jx]
                reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)
                    
                normalBalance = n_vertex[ix] + n_vertex[jx]
                normalBalance = normalBalance / np.linalg.norm(normalBalance)
                                
                if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
                    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
                        reflexNormal = -reflexNormal
                    
                    middlePoint = (v[ix] + v[jx]) / 2
                    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
                    origin = np.array([0, 0, 0])
                    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D)
                    
                    #Use projection point instead of D-component
                    #xcPair = [ix, jx, reflexNormal.tolist(), projection.tolist(), i, j]
                    xcPair = [ix, jx, reflexNormal, projection, i, j]
                    lxPairs.append(xcPair)
    
    return lxPairs

In [14]:
start = datetime.now()

lRfxPairs = baseQReflection(oVertex) #v_indices
print(len(lRfxPairs))

end = datetime.now()
print(end - start)

843
0:00:00.681803


In [15]:
def correctDirection(nVector):
    nVector = nVector * np.sign(nVector[np.argmax(abs(nVector))])
    return nVector

def normalized(X):
    return X / np.linalg.norm(X)

def projectPointLine(point, line, pointLine):
    ap = point - pointLine
    ab = line
    return pointLine + np.dot(ap, ab)/np.dot(ab, ab) * ab

def distancePointLine(point, line, pointLine):
    return np.linalg.norm(point - projectPointLine(point, line, pointLine))

def planesIntersection(pairM, pairN):
    U = normalized(np.cross(pairM[2], pairN[2]))
    M = np.array((pairM[2], pairN[2], U))
    X = np.array((np.dot(pairM[2], pairM[3]), np.dot(pairN[2], pairN[3]), 0.))
    
    origin = np.array([0, 0, 0])
    return correctDirection(U), projectPointLine(origin, U, np.linalg.solve(M, X))

def reflectPointLine(point, line, pointLine):
    projection = projectPointLine(point, line, pointLine)
    return 2 * projection - point

def reflectPointPlane(point, line, pointLine):
    projection = projectPointPlane(point, line, pointLine)
    return 2 * projection - point

In [16]:
def BaseContinuousQRotation(pairs):
    lrPairs = []    
    avg_k = []
    #Average curvatures
    for pair in pairs:
        avg_k.append([(k1_c[pair[4]] + k1_c[pair[5]])/2, (k2_c[pair[4]] + k2_c[pair[5]])/2])
    kt_rt = KDTree(avg_k)
    
    lk0, lk1 = [row[0] for row in avg_k], [row[1] for row in avg_k]
    th0 = np.percentile(lk0, 99.5) - np.percentile(lk0, 0.5)
    th1 = np.percentile(lk1, 99.5) - np.percentile(lk1, 0.5)
    #th0, th1 = max(lk0) - min(lk0), max(lk1) - min(lk1)
    rd = round(np.sqrt(th0**2 + th1**2) / 16, 3)
    
    for m in range(0, len(pairs) - 1):
        rpM = pairs[m]
        idx = kt_rt.query_ball_point([(k1_c[rpM[4]] + k1_c[rpM[5]])/2, 
                           (k2_c[rpM[4]] + k2_c[rpM[5]])/2], r = rd, return_sorted=True)
        idl = [kx for kx in idx if kx > m]
                
        for n in idl: #range(m + 1, len(pairs)):
            rpN = pairs[n]
            ratio_k1 = (k1_c[rpM[4]] + k1_c[rpM[5]]) / (k1_c[rpN[4]] + k1_c[rpN[5]])
            ratio_k2 = (k2_c[rpM[4]] + k2_c[rpM[5]]) / (k2_c[rpN[4]] + k2_c[rpN[5]])
            nparallel = abs(np.dot(rpM[2], rpN[2])) < TH_RATIO_K
            
            if TH_RATIO_K < ratio_k1 < 1/TH_RATIO_K and TH_RATIO_K < ratio_k2 < 1/TH_RATIO_K and nparallel:
                #print(m, rpM[2], n, rpN[2])
                line = planesIntersection(rpM, rpN)
                
                p1_i = (v[rpM[0]] + v[rpM[1]]) / 2
                p2_i = (v[rpN[0]] + v[rpN[1]]) / 2
                dist_1i = distancePointLine(p1_i, line[0], line[1]) #distance
                proj_1i = projectPointLine(p1_i, line[0], line[1]) #projection
                
                dist_2i = distancePointLine(p2_i, line[0], line[1]) #distance
                proj_2i = projectPointLine(p2_i, line[0], line[1]) #projection
                
                if np.linalg.norm(proj_1i - proj_2i) < CLUSTER_EP[0] and dist_1i - dist_2i < CLUSTER_EP[0]:
                    #line[0], line[1]
                    rtPair = [rpM[0:2], rpN[0:2], line[0], line[1], rpM[4:6], rpN[4:6], m, n]
                    lrPairs.append(rtPair)
    return lrPairs

In [17]:
start = datetime.now()

lcrPairs = BaseContinuousQRotation(lRfxPairs)

end = datetime.now()
print(end - start)

0:00:02.102941
